In [2]:
from filter_overlay import FilterOverlay

filterOverlay = FilterOverlay()
outp_path = filterOverlay.get_path('3', 'moustache_hitler')
print(outp_path)

load checkpoint from local path: ./mmpose/input/hrnetv2_w18_aflw_256x256-f2bbc62b_20210125.pth
./output/3.jpg
